# Project Shark Attacks

link: http://www.sharkattackfile.net/incidentlog.htm

In [338]:
import pandas as pd
import numpy as np
import re

In [339]:
!pwd

/Users/aldreybaldovi/Desktop/ironhack/labs/module_1/week_8/project_shark_attacks


In [340]:
df = pd.read_excel('GSAF5.xls', encoding='latin1')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2020.09.20,20-Sep-2020,2020.0,Unprovoked,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,Andrew Charles Eddy,M,...,Bull shark,"Click Orlando, 9/20/2020",2020.09.20-Eddy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.20,2020.09.20,6576.0,NaN,NaN
1,2020.09.17,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,Fraser Island,Fishing,male,M,...,"""whitetip shark""","K. McMurray, Tracking Sharks.com and B. Myatt,...",2020.09.17-FraserIsland.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.17,2020.09.17,6575.0,NaN,NaN
2,2020.09.16.b,16-Sep-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,Cabarita Beach,Foil-boarding,Christian Bungate,M,...,White shark,"B. Myatt, GSAF",2020.09.16.b-Bungate.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.16.b,2020.09.16.b,6574.0,NaN,NaN
3,2020.09.16,16-Sep-2020,2020.0,NaN,USA,Florida,"Daytona Beach Shores, Volusia County",Swimming,Eric Bowman,M,...,Shark involvement not confirmed,"K. McMurray, TrackingSharks.com",2020.09.16-Bowman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.16,2020.09.16,6573.0,NaN,NaN
4,2020.09.15.b,15-Sep-2020,2020.0,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",Surfing,Cole Smyth,M,...,NaN,"K. McMurray, TrackingSharks.com",2020.09.15.b-Smyth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.09.15.b,2020.09.15.b,6572.0,NaN,NaN


In [341]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [342]:
len(df)

25796

In [343]:
df_bkp = df.copy()

- Analisar a quantidade de vazios nas colunas de interesse ('Year', 'Sex', 'Country', 'Fatal (Y/N)')

In [344]:
df_bkp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8775 non-null   object 
 1   Date                    6575 non-null   object 
 2   Year                    6573 non-null   float64
 3   Type                    6566 non-null   object 
 4   Country                 6525 non-null   object 
 5   Area                    6108 non-null   object 
 6   Location                6025 non-null   object 
 7   Activity                6018 non-null   object 
 8   Name                    6360 non-null   object 
 9   Sex                     6003 non-null   object 
 10  Age                     3672 non-null   object 
 11  Injury                  6545 non-null   object 
 12  Fatal (Y/N)             6023 non-null   object 
 13  Time                    3154 non-null   object 
 14  Species                 3622 non-null 

# HYPOTHESIS: 
- There is more death cases with **man than woman** (SEX AND FATAL (Y/N) COLUMNS) **OK**
- There is more death cases with **man than woman** in all centuries **OK**
- Surf is the activity which causes more accident **in the last years** (YEAR AND ACTIVITY COLUMNS)
- Most of the cases were **unprovoked** (TYPE COLUMN)
- **Young people** is more likely to die by shark attacks (AGE COLUMN)
- The **location** with greater number of cases has no legislation or safety measures (COUNTRY AND AREA)
- Season with more cases is summer (DATE COLUMN AND **NEW COLUMN SEASON**)
- The most dangerous species of shark (SPECIES COLUMN) 

STEPS

- Use columns: 

`['Date', 'Year', 'Type', 'Country', 'Area', 'Sex ', 'Age','Species ','Activity','Fatal (Y/N)']`


- Drop columns: 

`['Case Number', 'Name', 'Location', 'Injury', 'Time','Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']`


- Rename columns (df = df.rename(columns = {'Date' : 'date'}) OR df.rename({'Date' : 'date'}, axis =1, inplace = True):
    - lower case
    - no spaces


- Rename similar words in lines:
    - **m:** male, M 
    - **f:** female, F 
    - **y:** Yes, Y, y 
    - **n:** No, no, N


- Analise columns with NAN:
    - Substitute NAN to 0 (`df.fillna(0)`) when necessary


# RESULTS

- Filtering Data Frame and renaming columns

In [345]:
# Filtering data frame with useful columns

df = df.loc[:,['Date', 'Year', 'Type', 'Country', 'Area', 'Sex ', 'Age','Species ','Activity','Fatal (Y/N)']]
df.head()

,Date,Year,Type,Country,Area,Sex,Age,Species,Activity,Fatal (Y/N)
0,20-Sep-2020,2020.0,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020.0,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020.0,Unprovoked,USA,Florida,M,15,NaN,Surfing,N


In [346]:
# Renaming columns

df.rename({'Date':'date', 'Year':'year', 'Type':'type', 'Country':'country', 'Area':'area', 
                 'Sex ':'sex', 'Age':'age','Species ':'species','Activity':'activity',
                 'Fatal (Y/N)':'fatal'}, axis =1, inplace = True)
df.head()

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020.0,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020.0,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020.0,Unprovoked,USA,Florida,M,15,NaN,Surfing,N


## There is more death cases with **man than woman** (DATE, YEAR, SEX AND FATAL (Y/N) COLUMNS)


### **Working with `date` and `year`:**

 - Delete null values
    
    - regex to use only the year
    
        `pattern = '\d\d\d\d'`
        
        `print(re.findall(pattern, text))`
        
    - group the number of fatal cases of the same year
    
        `fatal_cases_year = df.groupby('date', as_index = False)`

In [347]:
df['date'].value_counts()

1957           9
1942           8
1958           7
No date        6
1956           6
              ..
26-Mar-1951    1
Aug-1950       1
09-Apr-2002    1
27-Jul-1960    1
08-Mar-1942    1
Name: date, Length: 5685, dtype: int64

In [348]:
df['year'].value_counts()

2015.0    143
2017.0    139
2016.0    131
2011.0    128
2014.0    127
         ... 
1753.0      1
77.0        1
1785.0      1
1800.0      1
1543.0      1
Name: year, Length: 251, dtype: int64

In [349]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      6575 non-null   object 
 1   year      6573 non-null   float64
 2   type      6566 non-null   object 
 3   country   6525 non-null   object 
 4   area      6108 non-null   object 
 5   sex       6003 non-null   object 
 6   age       3672 non-null   object 
 7   species   3622 non-null   object 
 8   activity  6018 non-null   object 
 9   fatal     6023 non-null   object 
dtypes: float64(1), object(9)
memory usage: 2.0+ MB


In [350]:
null_lines = df['year'].isnull()
df[null_lines]

,date,year,type,country,area,sex,age,species,activity,fatal
444,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,M,35,Bull shark,Spearfishing,NaN
6351,Reported 19-Aug-1836,NaN,Unprovoked,ENGLAND,Cumberland,M,NaN,NaN,Swimming,Y
6575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
25791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [351]:
len(df[null_lines])

19223

In [352]:
drop_lines = list(df[null_lines].index)
df = df.drop(drop_lines, axis = 0)
len(df)

6573

In [353]:
zero = df.loc[(df['year'] == 0)]
zero

,date,year,type,country,area,sex,age,species,activity,fatal
6449,Ca. 214 B.C.,0.0,Unprovoked,NaN,Ionian Sea,M,NaN,NaN,Ascending from a dive,Y
6450,Ca. 336.B.C..,0.0,Unprovoked,GREECE,Piraeus,M,NaN,NaN,Washing his pig in preparation for a religious...,Y
6451,493 B.C.,0.0,Sea Disaster,GREECE,Off Thessaly,M,NaN,NaN,Shipwrecked Persian Fleet,Y
6452,Ca. 725 B.C.,0.0,Sea Disaster,ITALY,Tyrrhenian Sea,M,NaN,NaN,Shipwreck,Y
6453,After 2013,0.0,Unprovoked,AUSTRALIA,Queensland,M,NaN,Hammerhead shark,Spearfishing,N
...,...,...,...,...,...,...,...,...,...,...
6570,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Diving,Y
6571,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Pearl diving,Y
6572,1900-1905,0.0,Unprovoked,USA,North Carolina,M,NaN,NaN,Swimming,Y
6573,1883-1889,0.0,Unprovoked,PANAMA,NaN,M,NaN,NaN,NaN,Y


- **When year is equal 0, it is related to data from around 1900**
    - Before Century XIX: <= 1800
    - Century XIX: 1801 to 1900
    - Century XX: 1901 to 2000
    - Century XXI: 2001 to 2100

In [354]:
df['year'] = df['year'].astype('int')

In [355]:
df

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
2,16-Sep-2020,2020,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N
...,...,...,...,...,...,...,...,...,...,...
6570,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Diving,Y
6571,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,M,NaN,NaN,Pearl diving,Y
6572,1900-1905,0,Unprovoked,USA,North Carolina,M,NaN,NaN,Swimming,Y
6573,1883-1889,0,Unprovoked,PANAMA,NaN,M,NaN,NaN,NaN,Y


In [356]:
#Test if the column has null value

df['year'].isnull().value_counts()

False    6573
Name: year, dtype: int64

In [357]:
#to update zero values to 1900 in column `year`
df['year'].loc[(df['year'] == 0)] = 1800

/Users/aldreybaldovi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [358]:
df['year'].value_counts()

2015    143
2017    139
2016    131
2011    128
1800    127
       ... 
1822      1
1834      1
1859      1
77        1
500       1
Name: year, Length: 250, dtype: int64

**Try to find the 4 last numbers in date and replace it after:**

In [359]:
'''df['date'] = df['date'].astype('str')'''

"df['date'] = df['date'].astype('str')"

In [360]:
'''text = [i for i in df['date']]
pattern = '\d\d\d\d$'
df['date_2'] = [re.findall(pattern, text)]
df.head()'''

"text = [i for i in df['date']]\npattern = '\\d\\d\\d\\d$'\ndf['date_2'] = [re.findall(pattern, text)]\ndf.head()"

### **Working with `sex`:**

- Fatality cases per year according to sex:

In [361]:
def fill(x):
    '''
    Function to correct data from df['sex']
    '''
    
    pattern_F = '.*[fF].*'
    pattern_M = '.*[mM].*'
    
    if x != x:
        return 'NaN'
    
    if len(re.findall(pattern_F, x)) == 0 and len(re.findall(pattern_M, x)) == 0:
        return 'NaN'

    
    text = re.sub(pattern_F, 'F', x)
    
    text = re.sub(pattern_M, 'M', text)
    
    
    
    return text

In [362]:
#test

print(fill(' M'))
print(fill('M '))
print(fill(' F'))
print(fill('F '))
print(fill('lli'))
print(fill('M x 2'))
print(fill(np.nan))


M
M
F
F
NaN
M
NaN


In [363]:
df['sex'].value_counts()

M        5299
F         695
M           2
N           2
.           1
M x 2       1
lli         1
Name: sex, dtype: int64

In [364]:
df['sex'].apply(fill)

0       M
1       M
2       M
3       M
4       M
       ..
6570    M
6571    M
6572    M
6573    M
6574    M
Name: sex, Length: 6573, dtype: object

In [365]:
df['sex'].value_counts()

M        5299
F         695
M           2
N           2
.           1
M x 2       1
lli         1
Name: sex, dtype: int64

### **Working with `fatal`:**

In [458]:
#Correcting data from df['fatal']

df['fatal'].fillna(0)
df['fatal'].loc[(df['fatal'] == ' Y')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'Y ')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'y')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'y ')] = 'Y'
df['fatal'].loc[(df['fatal'] == ' y')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'Y x 2')] = 'Y'
df['fatal'].loc[(df['fatal'] == 'F')] = 'Y'
df['fatal'].loc[(df['fatal'] == ' N')] = 'N'
df['fatal'].loc[(df['fatal'] == 'N ')] = 'N'
df['fatal'].loc[(df['fatal'] == '2017')] = 'NaN'
df['fatal'].loc[(df['fatal'] == 'M')] = 'NaN'
df['fatal'].loc[(df['fatal'] == 'nan')] = 'NaN'
df['fatal'].loc[(df['fatal'] == 'UNKNOWN')] = 'NaN'


'''df_fatal_year['fatal'].rename({' Y':'Y', 'Y ':'Y', 'y':'Y', 'y ':'Y', ' y':'Y', 'Y x 2':'Y', 'F':'Y',
                               ' N':'N', 'N ':'N', '2017':'UNKNOWN', 'M':'UNKNOWN', 'nan':'UNKNOWN'},
                               axis = 0, inplace = True)
df_fatal_year['fatal'].value_counts()
'''

/Users/aldreybaldovi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


"df_fatal_year['fatal'].rename({' Y':'Y', 'Y ':'Y', 'y':'Y', 'y ':'Y', ' y':'Y', 'Y x 2':'Y', 'F':'Y',\n                               ' N':'N', 'N ':'N', '2017':'UNKNOWN', 'M':'UNKNOWN', 'nan':'UNKNOWN'},\n                               axis = 0, inplace = True)\ndf_fatal_year['fatal'].value_counts()\n"

In [459]:
df['fatal'].value_counts()

N      4537
Y      1412
NaN     624
Name: fatal, dtype: int64

In [460]:
df_fatal_year = df.loc[:,['year', 'fatal', 'sex']]
df_fatal_year

,year,fatal,sex
0,2020,N,M
1,2020,N,M
2,2020,N,M
3,2020,N,M
4,2020,N,M
...,...,...,...
6570,1800,Y,M
6571,1800,Y,M
6572,1800,Y,M
6573,1800,Y,M


### **Answering the question:**

In [461]:
# How many fatal cases in general

fatal_cases = df.loc[df['fatal'] == 'Y']
print(fatal_cases.shape)

(1412, 10)


In [462]:
#How many fatal cases with **man**

sex_M = df.loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]
print(sex_M.shape)

(1194, 2)


In [463]:
#How many fatal cases with **woman**

sex_F = df.loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]
print(sex_F.shape)

(108, 2)


## There is more death cases with **man than woman** in all centuries (SEX, FATAL, DATE AND YEAR)


### **Separating intervals between centuries:**

In [464]:
df['fatal'] = df['fatal'].astype('str')

In [465]:
df_fatal_year.groupby('year', as_index = False).count()

,year,fatal,sex
0,5,1,1
1,77,1,1
2,500,1,1
3,1543,1,1
4,1554,1,1
...,...,...,...
245,2016,131,127
246,2017,139,134
247,2018,121,121
248,2019,111,108


In [466]:
#separating intervals between centuries

#Before century XIX
start_date_BXIX = 0
end_date_BXIX = 1800

#Century XIX
start_date_XIX = 1801
end_date_XIX = 1900

#Century XX
start_date_XX = 1901
end_date_XX = 2000

#Century XXI
start_date_XXI = 2001
end_date_XXI = 2020

#masks:
mask_BXIX = (df_fatal_year['year'] > start_date_BXIX) & (df_fatal_year['year'] <= end_date_BXIX)

mask_XIX = (df_fatal_year['year'] > start_date_XIX) & (df_fatal_year['year'] <= end_date_XIX)

mask_XX = (df_fatal_year['year'] > start_date_XX) & (df_fatal_year['year'] <= end_date_XX)

mask_XXI = (df_fatal_year['year'] > start_date_XXI) & (df_fatal_year['year'] <= end_date_XXI)


### **Answering the question:**

#### Before century XIX

In [467]:
# Before century XIX : Fatal cases
bxix = df_fatal_year.loc[mask_BXIX].groupby('fatal', as_index = False).count()
bxix.loc[:,['fatal', 'year']]

,fatal,year
0,N,72
1,NaN,14
2,Y,85


In [468]:
# Before century XIX : Fatal cases by sex (male)
(df_fatal_year.loc[mask_BXIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(74, 2)

In [469]:
# Before century XIX : Fatal cases by sex (female)
(df_fatal_year.loc[mask_BXIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(5, 2)

#### Century XIX

In [470]:
# Century XIX : Fatal cases
xix = df_fatal_year.loc[mask_XIX].groupby('fatal', as_index = False).count()
xix.loc[:,['fatal', 'year']]

,fatal,year
0,N,223
1,NaN,81
2,Y,278


In [471]:
# Century XIX : Fatal cases : Fatal cases by sex (male)
(df_fatal_year.loc[mask_XIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(256, 2)

In [472]:
# Century XIX : Fatal cases : Fatal cases by sex (female)
(df_fatal_year.loc[mask_XIX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(9, 2)

#### Century XX

In [473]:
# Century XX : Fatal cases
xx = df_fatal_year.loc[mask_XX].groupby('fatal', as_index = False).count()
xx.loc[:,['fatal', 'year']]

,fatal,year
0,N,2348
1,NaN,358
2,Y,864


In [474]:
# Century XX : Fatal cases : Fatal cases by sex (male)
(df_fatal_year.loc[mask_XX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(713, 2)

In [475]:
# Century XX : Fatal cases : Fatal cases by sex (female)
(df_fatal_year.loc[mask_XX].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(72, 2)

#### Century XXI

In [476]:
# Century XXI : Fatal cases
xxi = df_fatal_year.loc[mask_XXI].groupby('fatal', as_index = False).count()
xxi.loc[:,['fatal', 'year']]

,fatal,year
0,N,1809
1,NaN,164
2,Y,175


In [477]:
# Century XXI : Fatal cases : Fatal cases by sex (male)
(df_fatal_year.loc[mask_XXI].loc[(df['fatal'] == 'Y') & (df['sex'] == 'M'),['fatal','sex']]).shape

(143, 2)

In [478]:
# Century XXI : Fatal cases : Fatal cases by sex (female)
(df_fatal_year.loc[mask_XXI].loc[(df['fatal'] == 'Y') & (df['sex'] == 'F'),['fatal','sex']]).shape

(22, 2)

In [479]:
bxix_y = bxix['year'][bxix['fatal']=='Y']
xix_y = xix['year'][xix['fatal']=='Y']
xx_y = xx['year'][xx['fatal']=='Y']
xxi_y = xxi['sex'][xxi['fatal']=='Y']

print(f'Before Century XIX: {bxix_y}, Century XIX: {xix_y}, Century XX: {xx_y}, Century XXI: {xxi_y}')

Before Century XIX: 2    85
Name: year, dtype: int64, Century XIX: 2    278
Name: year, dtype: int64, Century XX: 2    864
Name: year, dtype: int64, Century XXI: 2    165
Name: sex, dtype: int64


## **Young people** is more likely to die by shark attacks (AGE COLUMN)


### **Working with `age` column**

In [480]:
df['age'].head

<bound method NDFrame.head of 0        30
1       50s
2       NaN
3        48
4        15
       ... 
6570    NaN
6571    NaN
6572    NaN
6573    NaN
6574     15
Name: age, Length: 6573, dtype: object>

In [481]:
df['age'].value_counts()

19                 86
16                 86
18                 82
17                 82
17                 80
                   ..
MAKE LINE GREEN     1
30 or 36            1
36 & 26             1
74                  1
1                   1
Name: age, Length: 236, dtype: int64

In [482]:
null_lines_age = df['age'].isnull()
df[null_lines_age].head()

,date,year,type,country,area,sex,age,species,activity,fatal
2,16-Sep-2020,2020,Unprovoked,AUSTRALIA,New South Wales,M,NaN,White shark,Foil-boarding,N
9,02-Sep-2020,2020,Unprovoked,USA,Florida,M,NaN,"Nurse shark, juvenile",NaN,N
10,30-Aug-2020,2020,Watercraft,AUSTRALIA,New South Wales,NaN,NaN,White shark,Fishing / Filming sharks feeding on whale carcass,N
12,28-Aug-2020,2020,Unprovoked,USA,California,M,NaN,White shark,Kayak fishing,N
16,19-Aug-2020,2020,NaN,USA,South Carolina,F,NaN,NaN,Wading,N


In [483]:
len(df[null_lines_age])

2902

In [484]:
drop_lines_age = list(df[null_lines_age].index)

In [485]:
# Dropping null lines

df_age = df.drop(drop_lines_age, axis = 0)
len(df_age)

3671

In [486]:
df_age['age'].value_counts()

16                86
19                86
17                82
18                82
17                80
                  ..
16 to 18           1
21, 34,24 & 35     1
86                 1
18 to 22           1
1                  1
Name: age, Length: 236, dtype: int64

In [487]:
df_age['age'] = df_age['age'].astype('str')

In [488]:
def fix_age(x):
    '''
    Function to correct data from df['age']
    
    This function takes the last numbers of the text
    '''
    
    pattern = '\d?\d$'
    
    if x != x:
        return '0'
    
    if len(re.findall(pattern, x)) == 0: 
        return '0'
    
    text = re.findall(pattern, x)[0]
    
   
    
    return text

In [489]:
#test

fix_age('18 e 12 e 03543350s')

'0'

In [490]:
df_age

,date,year,type,country,area,sex,age,species,activity,fatal
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,50s,"""whitetip shark""",Fishing,N
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N
5,15-Sep-2020,2020,Unprovoked,USA,Florida,M,25,NaN,Surfing,N
...,...,...,...,...,...,...,...,...,...,...
6516,1960s,1800,Unprovoked,IRAQ,Basrah,M,16,Bull shark,Swimming in section of river used for washing ...,N
6527,Before 1957,1800,Provoked,CUBA,Havana Province,M,50,NaN,"Shark fishing, knocked overboard",Y
6538,1941-1942,1800,Unprovoked,IRAQ,Basrah,M,13 or 14,Bull shark,Swimming,Y
6549,Early 1930s,1800,Unprovoked,BELIZE,NaN,M,16,12' tiger shark,Standing,Y


In [491]:
# Applying the function fix_age in df_age['age']

df_age['age'] = df_age['age'].apply(fix_age)

In [492]:
df_age['age'].value_counts()

17    162
18    157
19    147
20    146
16    144
     ... 
86      1
82      1
72      1
87      1
81      1
Name: age, Length: 82, dtype: int64

In [493]:
df_age['age'] = df_age['age'].astype('int')

In [494]:
# # Defining age groups:

# #kids = 0 to 12
# mask_kids = (df_age['age'] > 0) & (df_age['age'] <= 12)

# #teenager = 13 to 17
# mask_teenager = (df_age['age'] > 12) & (df_age['age'] <= 17)

# #young_adult = 18 to 32
# mask_young_adult = (df_age['age'] > 17) & (df_age['age'] <= 32)

# #adult = 33 to 55
# mask_adult = (df_age['age'] > 32) & (df_age['age'] <= 55)

# #older > 55
# mask_older = (df_age['age'] > 55)


In [495]:
# Defining age groups:
df_age['age_group'] = pd.cut(df_age['age'],[0,12,17,32,55,200], labels = ['kids','teenager','young_adult','adult','older'])
df_age.head()

,date,year,type,country,area,sex,age,species,activity,fatal,age_group
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N,young_adult
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,0,"""whitetip shark""",Fishing,N,NaN
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N,adult
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N,teenager
5,15-Sep-2020,2020,Unprovoked,USA,Florida,M,25,NaN,Surfing,N,young_adult


In [503]:
# Before century XIX : Verifying the number of fatal cases by age group
df_age.loc[mask_BXIX,'century'] = 18
df_age.loc[mask_XIX,'century'] = 19
df_age.loc[mask_XX,'century'] = 20
df_age.loc[mask_XXI,'century'] = 21


df_age


,date,year,type,country,area,sex,age,species,activity,fatal,age_group,century
0,20-Sep-2020,2020,Unprovoked,USA,Florida,M,30,Bull shark,Snorkeling,N,young_adult,21
1,17-Sep-2020,2020,Provoked,AUSTRALIA,Queensland,M,0,"""whitetip shark""",Fishing,N,NaN,21
3,16-Sep-2020,2020,NaN,USA,Florida,M,48,Shark involvement not confirmed,Swimming,N,adult,21
4,15-Sep-2020,2020,Unprovoked,USA,Florida,M,15,NaN,Surfing,N,teenager,21
5,15-Sep-2020,2020,Unprovoked,USA,Florida,M,25,NaN,Surfing,N,young_adult,21
...,...,...,...,...,...,...,...,...,...,...,...,...
6516,1960s,1800,Unprovoked,IRAQ,Basrah,M,16,Bull shark,Swimming in section of river used for washing ...,N,teenager,18
6527,Before 1957,1800,Provoked,CUBA,Havana Province,M,50,NaN,"Shark fishing, knocked overboard",Y,adult,18
6538,1941-1942,1800,Unprovoked,IRAQ,Basrah,M,14,Bull shark,Swimming,Y,teenager,18
6549,Early 1930s,1800,Unprovoked,BELIZE,NaN,M,16,12' tiger shark,Standing,Y,teenager,18


In [514]:
df_grouped = df_age.groupby(['fatal','age_group','century','sex']).agg(count_cases = ('date','count')).reset_index()
#df_grouped.loc[:,['fatal','age_group','century','sex']]

df_grouped

,fatal,age_group,century,sex,count_cases
0,N,kids,18,F,0
1,N,kids,18,M,1
2,N,kids,18,M,0
3,N,kids,18,lli,0
4,N,kids,19,F,0
...,...,...,...,...,...
235,Y,older,20,lli,0
236,Y,older,21,F,7
237,Y,older,21,M,14
238,Y,older,21,M,0


In [505]:
import gc
gc.collect()

40

In [517]:
# Verifying the number of fatal cases by age by sex by century
df_grouped.query('fatal == "Y"')

,fatal,age_group,century,sex,count_cases
160,Y,kids,18,F,0
161,Y,kids,18,M,0
162,Y,kids,18,M,0
163,Y,kids,18,lli,0
164,Y,kids,19,F,0
...,...,...,...,...,...
235,Y,older,20,lli,0
236,Y,older,21,F,7
237,Y,older,21,M,14
238,Y,older,21,M,0


## Surf is the activity which causes more accident **in the last years** (YEAR AND ACTIVITY COLUMNS)


- Grouping the activities which shows the greatest number of fatalities

In [328]:
df_fatal_act = df.loc[:,['activity', 'fatal']]
df_fatal_act

,activity,fatal
0,Snorkeling,N
1,Fishing,N
2,Foil-boarding,N
3,Swimming,N
4,Surfing,N
...,...,...
6570,Diving,Y
6571,Pearl diving,Y
6572,Swimming,Y
6573,NaN,Y


In [330]:
df_fatal_act.groupby('activity', as_index = False).count()

,activity,fatal
0,,1
1,,1
2,Diving for abalone (Hookah),1
3,a canoe was pursuing a schooner that had forc...,1
4,"""Boat accident""",1
...,...,...
1547,ship torpedoed 400 miles off the African coas....,1
1548,small boat,1
1549,wreck of the State Oil Company ship Permina,1
1550,yachting accident,1


## Most of the cases were **unprovoked** (TYPE COLUMN)


## The **location** with greater number of cases has no legislation or safety measures (COUNTRY AND AREA)


In [137]:
df.groupby('country', as_index = False).count()

,country,date,year,type,area,sex,age,species,activity,fatal
0,PHILIPPINES,1,1,1,1,1,0,0,1,1
1,TONGA,3,3,3,3,3,1,3,3,3
2,ADMIRALTY ISLANDS,1,1,1,1,1,1,0,0,1
3,AFRICA,1,1,1,0,1,0,0,1,1
4,ALGERIA,1,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
209,VIETNAM,15,15,14,10,15,6,4,12,15
210,WEST INDIES,2,2,2,0,2,0,0,2,2
211,WESTERN SAMOA,1,1,1,1,1,1,1,1,1
212,YEMEN,2,2,2,2,1,0,1,2,2


## Season with more cases is summer (DATE COLUMN AND **NEW COLUMN SEASON**)


## The most dangerous species of shark (SPECIES COLUMN) 